In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import re
from transformers import pipeline

In [7]:
def extract_numbers(text, given_strings):
    # Split text into a list of words
    words = text.split()
    
    # Find the indices of the given strings in the list of words
    indices = [i for i, word in enumerate(words) if any(s in word for s in given_strings)]
    
    # Initialize an empty list to store the numbers
    numbers = []
    
    # Loop through each index
    for index in indices:
        # Define the range of words to search for numbers
        start = max(index - 1, 0)
        end = min(index + 2, len(words))
        
        # Extract the words within the range
        context = words[start:end]
        
        # Find all numbers in the context
        context_numbers = [int(re.sub('[^0-9]+', '', word)) for word in context if re.sub('[^0-9]+', '', word).isdigit()]
        
        # Add the numbers to the list
        numbers.extend(context_numbers)
    
    return numbers


In [8]:
def get_text_and_values(text, key_list):
  sentences = re.split("(?<=[.?!])\s+(?=[A-Za-z])", text)
  total_numbers= []
  infoDict = {}
  for sentence in sentences:
    numbers = extract_numbers(text = sentence, given_strings = key_list)
    total_numbers.append(numbers)
    if not numbers:
      continue
    else: infoDict[sentence] = numbers
  return infoDict

In [9]:
def get_useful_text(dictionary):
  keysList = list(dictionary.keys())
  prova = ('\n'.join(keysList))
  return prova

In [10]:
def get_values(dictionary):
  pr = list(dictionary.values())
  return pr

In [11]:
def initialize_qa_transformer(model):
  qa = pipeline("text2text-generation", model=model)
  return qa

In [12]:
def get_answers_unfiltered(dictionary):
  keysList = list(dictionary.keys())
  answers = []
  for kl in keysList:
    answer = qa(kl+" Domanda: Quanti metri quadri misura l'immobile")
    answers.append(answer)
  return answers

In [13]:
def get_answers_filtered(dictionary):
  keysList = list(dictionary.keys())
  previous_answer = None
  answers = []
  for kl in keysList:
    answer = qa(kl+" Domanda: Quanti metri quadri misura l'immobile")
    if answer != previous_answer:
      answers.append(answer)
    previous_answer = answer
  return answers

In [26]:
def get_total(answered_values, text, keywords):
  numbers = [int(x[0]['generated_text']) for x in answered_values if x[0]['generated_text'].isdigit()]
  total = 0
  sum = 0
  total_list = []
# Define a regular expression pattern that will match a number
  pattern = r'\d+'
# Loop through the keywords and search for them in the text
  found = False
  for keyword in keywords:
    # Build a regular expression pattern that looks for the keyword
    # followed by up to three words, then a number
      keyword_pattern = f'{keyword}(\\s+\\w+){{0,3}}\\s+({pattern})'
      match = re.search(keyword_pattern, text, re.IGNORECASE)
      if match:
        # If we find a match, print the number and set found to True
          number = match.group(2)
          if number in numbers:
            total_list.append(int(number))
            print(f"Found a value ({number}) for keyword '{keyword}'.")
            found = True  

# If we didn't find a match
  if not found:
    for value in numbers:
      total += value
    total_list.append(total)
 #If there is more than one total, it means different lots with many total measures for each house. Calculate the sum of the totals mq  
  for value in total_list:
    sum += value
  return sum


**Prova**

Inizializzo var con il documento:

In [15]:
#@title

house = """Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
1 
 
 
 
TRIBUNALE ORDINARIO DI ROMA 
SEZ. IV – ESEC. IMMOBILIARI – G.E. DOTT. Romolo Ciufolini 
PROCEDURA ESECUTIVA N. 1668/2016 R.G.E. 
Custode Giudiziario I.V.G. di Roma tel 06/83751500,  
Mail: ivgroma@associazioneivg.legalmail.it. 
 
AVVISO DI VENDITA DI BENI IMMOBILI 
 
II ESPERIMENTO  
  
Il Dott. Carlo Gaddi, Notaio in Roma Professionista Delegato (referente della procedura) dal 
Tribunale di Roma con ordinanza in data 12/09/2019 
 
AVVISA CHE 
 
presso la sala aste del gestore della vendita ZUCCHETTI SOFTWARE GIURIDICO SRL, sita in 
Roma, Via Lucrezio Caro n° 63 e tramite il portale dello stesso www.fallcoaste.it – il giorno 2 
febbraio 2021, alle ore 15,00 (quindici), avrà luogo la 
 
VENDITA SENZA INCANTO  
CON MODALITA' TELEMATICA ASINCRONA  
 
dei seguenti beni 
 
LOTTO 1 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido. Piazza Ener Bettica n. 26, e precisamente: 
- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una 
superficie di mq. 44 circa a confine con  Piazza Ener Bettica, negozio sub. 4, viale condominiale, 
salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1305, Sub. 5, Zc. 7, Categoria C1, cl. 8, mq. 27, Piazza Ener Bettica n° 26, P.T, int. 1, rendita catastale 
Euro 715,34. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 31575 del 1968 e Ultimo progetto n.4445 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n. 29630 del 1978 e progressivo 
certificato n. 174 del 10/07/1979.    
Per la regolarizzazione delle opere interne (in quanto esiste difformità catastale), sanabili ai sensi 
dell'art.36 D.P.R 380/01 occorrerà eseguire una Cila in Sanatoria con costi pari ad euro 2000,00 per 
onorario tecnico ed euro 1000,00 di Sanzione ed euro 251.24 per diritti di segreteria, nuovo 
accatastamento euro 550. Per quanto riguarda il ripristino dello stato dei luoghi occorreranno i 
seguenti costi in relazione alle opere da realizzare: 1)Demolizione e trasposto a discarica dei 
materiali di risulta : € 4.000,00 2)Ricostruzione di pareti interne: € 2.000,00 3)Impianto idrico-
sanitario: € 2.000,000 4)Impianto elettrico: € 2.000,00 5)Tinteggiature: € 2.000,00  
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli:  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
2 
 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità.  
 
al valore d’asta di Euro 56.000 (cinquantaseimila) 
offerta minima Euro 42.000 (quarantaduemila) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 3.000 ( tremila) 
LOTTO 2 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido. Piazza Ener Bettica n. 27/28, Via Carlo Bosio n° 117-119 e precisamente: 
- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una 
superficie di mq. 92 circa a confine con  Piazza Ener Bettica, negozio sub. 4, Via Carlo Bosio, salvo 
altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1305, Sub. 4, Zc. 7, Categoria C1, cl. 7, mq. 92, Piazza Ener Bettica n° 26-28 Via Carlo Bosio n° 117-
119 P.T, int. 2, rendita catastale Euro 2.096,37. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 31575 del 1968 e Ultimo progetto n.4445 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.29630 del 1978 e progressivo 
certificato n. 174 del 10/07/1979.    
Per la regolarizzazione delle opere interne sanabili ai sensi dell'art.36 D.P.R 380/01 occorrerà 
eseguire una Cila in Sanatoria con costi pari ad euro 2000,00 per onorario tecnico ed euro 1000,00 
di Sanzione ed euro 251.24 per diritti di segreteria. Per quanto riguarda il ripristino dello stato dei 
luoghi occorreranno i seguenti costi in relazione alle opere da realizzare: 1)Demolizione e trasposto 
a discarica dei materiali di risulta : € 6000,00 2)Ricostruzione di pareti interne: € 2.000,00 
3)Impianto idrico-sanitario: € 2.000,000 4)Impianto elettrico: € 3000,00 5)Tinteggiature: € 
4000,00  
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
3) Ipoteca giudiziale derivante da decreto ingiuntivo iscritta presso la Conservatoria di Roma 1 in 
data  21/10/2016 al n° 20443 di formalità.  
 
al valore d’asta di Euro 140.000 (centoquarantamila) 
offerta minima Euro 105.000 (centocinquemila) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 5.000 (cinquemila) 
LOTTO 3 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Carlo Bosio n° 109 e precisamente: 
- garage sito al piano S1 dell’edificio E, distinto con l’interno 22, della superficie di mq. 16, circa a 
confine con vano scale, rampa condominiale, salvo altri.  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
3 
 
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1305, Sub. 47, Zc. 7, Categoria C/6, cl. 3, mq. 16, Via Carlo Bosio n° 109, S1, int. 22, edificio “E”, 
rendita catastale Euro 96,68. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 31575 del 1968 e Ultimo progetto n.4445 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.29630 del 1978 e progressivo 
certificato n. 174 del 10/07/1979.    
 Stato di occupazione: occupato senza titolo per la procedura l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
  
al valore d’asta di Euro 7.500 (settemilacinquecento) 
offerta minima Euro 5.625 (cinquemilaseicentoventicinque) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 400 (quattrocento) 
LOTTO 4 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Carlo Bosio n° 107 e precisamente: 
- garage sito al piano S1 dell’edificio F, distinto con l’interno 22, della superficie di mq. 12, circa a 
confine con vano scale, rampa condominiale, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1307, Sub. 47, Zc. 7, Categoria C/6, cl. 3, mq. 11, Via Carlo Bosio n° 107, P. S1, int. 22, edificio “F”, 
rendita catastale Euro 66,47. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 31575 del 1968 e Ultimo progetto n.4445 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.29630 del 1978 e progressivo 
certificato n. 174 del 10/07/1979.    
 Stato di occupazione: occupato senza titolo per la procedura l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e nota di pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 5.000 (cinquemila) 
offerta minima Euro 3.750 (tremilasettecentocinquanta) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 300 ( trecento) 
LOTTO 5 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Ferdinando Acton n° 54 e precisamente: 
- cantina sita al piano S1 dell’edificio C, scala “B”, della superficie di mq. 58, circa a confine con corte 
condominiale su più lati, vano scale condominiale, salvo altri.  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
4 
 
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1100, Part. 
2, Sub. 45, Zc. 7, Categoria C/2, cl. 6, mq. 58, Via Ferdinado Acton n° 54, P. S1, scala B, edificio “C”, 
rendita catastale Euro 475,24. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 36276 del 1961 e Ultimo progetto n.58249 del 1962.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.45562 del 1965 e progressivo 
certificato n. 808 del 27/09/1966  
E stata realizzata una divisione illegittima pertanto andranno eseguiti lavori di ripristino per riportare lo 
stato dei luoghi alla situazione originaria di impianto 
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 18.480 (diciottomilaquattrocentottanta) 
offerta minima Euro 13.860 (tredicimilaottocentosessanta) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 500 (cinquecento) 
LOTTO 6 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Ferdinando Acton n° 54 e precisamente: 
- cantina sita al piano S1 dell’edificio C, scala “A”, della superficie di mq. 107, circa a confine con 
corte condominiale su più lati, vano scale, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1100, Part. 
2, Sub. 23, Zc. 7, Categoria C/2, cl. 6, mq. 107, Via Ferdinado Acton n° 54, P. S1, scala A, edificio “C”, 
rendita catastale Euro 257,61. 
 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 36276 del 1961 e Ultimo progetto n.58249 del 1962.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.45562 del 1965 e progressivo 
certificato n. 808 del 27/09/1966  
E stata realizzata una diversa distribuzione di spazi interni con realizzazione di un w.c.; pertanto andranno 
eseguiti lavori di ripristino per riportare lo stato dei luoghi alla situazione originaria di impianto.  
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 27.280 (ventisettemiladuecentottanta) 
offerta minima Euro 20.420 (ventimilaquattrocentoventi) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 2.000 (duemila) 
LOTTO 7 
 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
5 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Ferdinando Acton n° 56 e precisamente: 
- magazzino sito al piano S1 dell’edificio D, scala “A”, della superficie di mq. 114, circa a confine con 
corte condominiale su più lati, vano scale, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1100, Part. 
181, Sub. 23, Zc. 7, Categoria C/2, cl. 6, mq. 115, Via Ferdinado Acton n° 56, P. S1, scala A, edificio 
“D”, rendita catastale Euro 510,78. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 36276 del 1961 e Ultimo progetto n.58249 del 1962.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.45562 del 1965 e progressivo 
certificato n. 808 del 27/09/1966  
E stata realizzata una diversa distribuzione di spazi interni con realizzazione di un w.c.; pertanto andranno 
eseguiti lavori di ripristino per riportare lo stato dei luoghi alla situazione originaria di impianto.  
Stato di occupazione: l’immobile risulta occupato in forza di contratto di locazione del 19/12/1984 
rinnovatosi tacitamente e attualmente disdettato dal custode; 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 40.800 (quarantamilaottocento) 
offerta minima Euro 30.600 (trentamilaseicento) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 3.000 (tremila) 
LOTTO 8 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Ferdinando Acton n° 58 e precisamente: 
- magazzino sito al piano S1 dell’edificio E, scala “B”, della superficie di mq. 114, circa a confine con 
corte condominiale su più lati, vano scale, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1100, Part. 
6, Sub. 45, Zc. 7, Categoria C/2, cl. 6, mq. 106, Via Ferdinado Acton n° 58, P. S1, scala B, edificio “E”, 
rendita catastale Euro 470,80. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 36276 del 1961 e Ultimo progetto n.58249 del 1962.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.45562-B del 1965 e progressivo 
certificato n. 810 del 27/09/1966  
Stato di occupazione: l’immobile risulta occupato in forza di contratto di locazione del 12/02/1982 
rinnovatosi tacitamente e attualmente disdettato dal custode; 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 32.000 (tgrentaduemila) 
offerta minima Euro 24.000 (ventiquattromila) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 2.500 (duemilacinquecento) 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
6 
 
LOTTO 9 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Ferdinando Acton n° 58 e precisamente: 
- magazzino sito al piano S1 dell’edificio E, scala “A”, della superficie di mq. 58, circa a confine con 
corte condominiale  vano scale condominiale, corridoio d’accesso, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1100, Part. 
6, Sub. 23, Zc. 7, Categoria C/2, cl. 6, mq. 58, Via Ferdinado Acton n° 58, P. S1, scala A, edificio “E”, 
rendita catastale Euro 257,61. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 36276 del 1961 e Ultimo progetto n.58249 del 1962.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.45562-B del 1965 e progressivo 
certificato n. 810 del 27/09/1966  
E stata realizzata una porta in fondo al magazzino e le aperture finestrate non corrispondono; pertanto 
andranno eseguiti lavori di ripristino per riportare lo stato dei luoghi alla situazione originaria di impianto. 
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 17.200 (diciassettemiladuecento) 
offerta minima Euro 12.900 (dodicimilanovecento) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 1.000 (mille) 
LOTTO 10 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Danilo Stiepovich n° 129 e precisamente: 
- negozio sito al piano terra dell’edificio A, della superficie di mq. 53 circa a confine con Via Danilo 
Stiepovich, subalterni 1 e 509 della particella 1401, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1041, Sub. 508, Zc. 7, Categoria C/1, cl. 7, mq. 55, Via Danilo Stiepovich n° 129-131, P. T, rendita 
catastale Euro 1.252,67. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 32613 del 1968 e Ultimo progetto n.14910 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.1745 del 1980 e progressivo 
certificato n. 341 del 31/12/1981   
Il negozio necessita di lavori di ripristino in quanto accorpato senza titolo ad altri lotti della presente 
perizia Esiste difformità catastale: Muro perimetrale confinante col sub 509 inesistente diversa 
distribuzione interna.  
Risulta necessario il ripristino della parete di confine col sub 509 come da planimetria catastale agli atti. 
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 85.600 (ottantacinquemilaseicento) 
offerta minima Euro 64.200 (sessantaquattromiladuecento) 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
7 
 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 5.000 (cinquemila) 
LOTTO 11 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Danilo Stiepovich n° 133-135-137 e precisamente: 
- negozio sito al piano terra, della superficie di mq. 80 circa a confine con Via Danilo Stiepovich, 
subalterno 508 della particella 1401, Via Armando Crisciani, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1041, Sub. 509, Zc. 7, Categoria C/1, cl. 7, mq. 70, Via Danilo Stiepovich n° 133-135-137, Via 
Armando Crisciani, P. T, rendita catastale Euro 1.594,30. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 32613 del 1968 e Ultimo progetto n.14910 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.1745 del 1980 e progressivo 
certificato n. 341 del 31/12/1981   
Il negozio necessita di lavori di ripristino in quanto accorpato senza titolo ad altri lotti della presente 
perizia Esiste difformità catastale: Muro perimetrale confinante col sub 508 inesistente diversa 
distribuzione interna.  
Risulta necessario il ripristino della parete di confine col sub 508 e ripristino del bagno ed antibagno. 
come da planimetria catastale agli atti. 
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 123.200 (centoventitremiladuecento) 
offerta minima Euro 92.400 (novantaduemilaquattrocento) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 5.000 (cinquemila) 
LOTTO 12 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Armando Crisciani n° 22/24 e precisamente: 
- negozio sito al piano terra dell’edificio “A”, della superficie di mq. 50 circa a confine con subalterni 
509 e 511 della particella 1401, Via Armando Crisciani, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1041, Sub. 510, Zc. 7, Categoria C/1, cl. 7, mq. 50, Via Armando Crisciani 22-24, P. T, rendita 
catastale Euro 1.138,79. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 32613 del 1968 e Ultimo progetto n.14910 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n. 1745 del 1980 e progressivo 
certificato n. 341 del 31/12/1981   
Il negozio necessita di lavori di ripristino in quanto accorpato senza titolo ad altri lotti della presente 
perizia Esiste difformità catastale risulta necessario il ripristino della parete di confine con il sub 511 e 
con il sub 509 come da planimetria agli atti 
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
8 
 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 72.000 (settantaduemila) 
offerta minima Euro 54.000 (cinquantaquattromila) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 3.500 (tremilacinquecento) 
LOTTO 13 
 
Quota pari a 1/1 del diritto di piena proprietà della porzione immobiliare sita a Roma (RM), 
località ostia Lido, Via Armando Crisciani n° 20 e precisamente: 
- negozio sito al piano terra dell’edificio “A”, della superficie di mq. 61 circa a confine con subalterni 
6 e 510 della particella 1401, Via Armando Crisciani, salvo altri.  
Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. 
1041, Sub. 511, Zc. 7, Categoria C/1, cl. 7, mq. 64, Via Armando Crisciani 20, P. T, rendita catastale 
Euro 1.457,65. 
Il fabbricato di cui fa parte la porzione immobiliare in oggetto è stato realizzato con regolare concessione 
edilizia con Primo progetto n. 32613 del 1968 e Ultimo progetto n.14910 del 1975.  
E' stato rilasciato il certificato di Agibiltà con protocollo richiesta n.1745 del 1980 e progressivo 
certificato n. 341 del 31/12/1981   
Il negozio necessita di lavori di ripristino  
Stato di occupazione: l’immobile è libero 
Formalità pregiudizievoli: 
1) Pignoramento Trascritto a Agenzia Entrate Ufficio Provinciale di Roma 1 in data 10/10/2016 al 
n° 74903 di formalità e pignoramento in rettifica dell’11/11/2016 al n° 86629 di formalità.  
2) Ipoteca volontaria derivante da mutuo fondiario Iscritta presso la Conservatoria di Roma 1 in 
data  13/08/2004 al n° 26.637 di formalità. 
 
al valore d’asta di Euro 92.000 (novantaduemila) 
offerta minima Euro 69.000 (sessantanovemila) 
 
In caso di gara ex art. 573 cpc eventuali offerte in aumento non potranno essere inferiori a: 
Euro 5.000 (cinquemila) 
 
D E T E R M I N A 
le seguenti modalità di presentazione delle offerte e di svolgimento della procedura di vendita. 
 
[A] 
 
DISCIPLINA DELLA VENDITA SENZA INCANTO CON MODALITA’ ASINCRONA 
 
1. MODALITÀ DI PRESENTAZIONE DELL’OFFERTA TELEMATICA 
Le offerte andranno obbligatoriamente redatte e depositate con le modalità ed i contenuti tutti 
previsti dagli artt. 12 e seguenti del Decreto del Ministro della Giustizia n. 32 del 26 febbraio 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
9 
 
2015 1 
 
1                                                                                       Art. 12 
MODALITÀ DI PRESENTAZIONE DELL'OFFERTA E DEI DOCUMENTI ALLEGATI 
1. L'offerta per la vendita telematica deve contenere:  
a) i dati identificativi dell'offerente, con l'espressa indicazione del codice fiscale o della partita IVA;  
b) l'ufficio giudiziario presso il quale pende la procedura;  
c) l'anno e il numero di ruolo generale della procedura;  
d) il numero o altro dato identificativo del lotto;  
e) la descrizione del bene;  
f) l'indicazione del referente della procedura;  
g) la data e l'ora fissata per l'inizio delle operazioni di vendita;  
h) il prezzo offerto e il termine per il relativo pagamento, salvo che si tratti di domanda di partecipazione 
all'incanto;  
i) l'importo versato a titolo di cauzione;  
l) la data, l'orario e il numero di CRO del bonifico effettuato per il versamento della cauzione;  
m) il codice IBAN del conto sul quale è stata addebitata la somma oggetto del bonifico di cui alla lettera l);  
n) l'indirizzo della casella di posta elettronica certificata di cui al comma 4 o, in alternativa, quello di cui al 
comma 5, utilizzata per trasmettere l'offerta e per ricevere le comunicazioni previste dal presente regolamento;  
o) l'eventuale recapito di telefonia mobile ove ricevere le comunicazioni previste dal presente regolamento.  
2. Quando l'offerente risiede fuori dal territorio dello Stato, e non risulti attribuito il codice fiscale, si deve indicare il 
codice fiscale rilasciato dall’autorità fiscale del Paese di residenza o, in mancanza, un analogo codice identificativo, 
quale ad esempio un codice di sicurezza sociale o un codice identificativo. In ogni caso deve essere anteposto il 
codice del paese assegnante, in conformità alle regole tecniche di cui allo standard ISO 3166-1 alpha-2code 
dell'International Organization for Standardization. 
3. L'offerta per la vendita telematica è redatta e cifrata mediante un software realizzato dal Ministero, in forma di 
documento informatico privo di elementi attivi e in conformità alle specifiche tecniche di cui all'articolo 26 del 
presente decreto. Il software di cui al periodo precedente è messo a disposizione degli interessati da parte del gestore 
della vendita telematica e deve fornire in via automatica i dati di cui al comma 1, lettere b), c), d), e), f) e g), nonché' 
i riferimenti dei gestori del servizio di posta elettronica certificata per la vendita telematica iscritti a norma 
dell'articolo 13, comma 4. 
4. L'offerta è trasmessa mediante la casella di posta elettronica certificata per la vendita telematica. La trasmissione 
sostituisce la firma elettronica avanzata dell'offerta, sempre che l'invio sia avvenuto richiedendo la ricevuta completa 
di avvenuta consegna di cui all'articolo 6, comma 4 del decreto del Presidente della Repubblica, 11 febbraio 2005, n. 
68 e che il gestore del servizio di posta elettronica certificata attesti nel messaggio o in un suo allegato di aver 
rilasciato le credenziali di accesso in conformità a quanto previsto dall'articolo 13, commi 2 e 3. Quando l'offerta è 
formulata da più persone alla stessa deve essere allegata la procura rilasciata dagli altri offerenti al titolare della 
casella di posta elettronica certificata per la vendita telematica. La procura è redatta nelle forme dell'atto pubblico o 
della scrittura privata autenticata e può essere allegata anche in copia per immagine. 
5. L'offerta, quando è sottoscritta con firma digitale, può essere trasmessa a mezzo di casella di posta elettronica 
certificata 
anche 
priva 
dei 
requisiti 
di 
cui 
all'articolo 
2, 
comma 
1, 
lettera 
n). 
Si applica il comma 4, terzo periodo, e la procura è rilasciata a colui che ha sottoscritto l'offerta a norma del presente 
comma. 
6. I documenti sono allegati all'offerta in forma di documento informatico o di copia informatica, anche per 
immagine, privi di elementi attivi. I documenti allegati sono cifrati mediante il software di cui al comma 3. Le 
modalità di congiunzione mediante strumenti informatici dell'offerta con i documenti alla stessa allegati sono fissate 
dalle specifiche tecniche di cui all'articolo 26. 
Art. 13 
MODALITÀ DI TRASMISSIONE DELL'OFFERTA  
1. L'offerta e i documenti allegati sono inviati a un apposito indirizzo di posta elettronica certificata del Ministero 
mediante la casella di posta elettronica certificata indicata a norma dell'articolo 12, comma 1, lettera n).  
2. Ciascun messaggio di posta elettronica certificata per la vendita telematica contiene, anche in un allegato, 
l'attestazione del gestore della casella di posta elettronica certificata per la vendita telematica di aver provveduto al 
rilascio delle credenziali previa identificazione del richiedente a norma del presente regolamento.  
3. Quando l'identificazione è eseguita per via telematica, la stessa può aver luogo mediante la trasmissione al gestore 
di cui al comma 1 di una copia informatica per immagine, anche non sottoscritta con firma elettronica, di un 
documento analogico di identità del richiedente. La copia per immagine è priva di elementi attivi ed ha i formati 
previsti dalle specifiche tecniche stabilite a norma dell'articolo 26. Quando l'offerente non dispone di un documento 
di identità rilasciato da uno dei Paesi dell'Unione europea, la copia per immagine deve essere estratta dal passaporto.  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
10 
 
Anche ad integrazione di quanto previsto nel citato decreto le offerte di acquisto (in bollo2) 
potranno essere presentate unicamente dall’offerente (o da uno degli offerenti o dal legale 
rappresentante della società offerente) o dal procuratore legale anche a norma dell’art.579 
ultimo co. c.p.c., sino alle ore 23.59 del giorno precedente la data fissata per il loro esame e per 
la vendita, tramite il modulo web “Offerta telematica” fornito dal Ministero della Giustizia 
(accessibile dalla scheda del lotto in vendita presente sul portale del gestore della vendita 
telematica), secondo le indicazioni riportate nel “Manuale Utente” pubblicato sul portale delle 
vendite pubbliche del Ministero della Giustizia; il file criptato in formato zip .p7m, contenente 
l’offerta integrale e gli allegati all’offerta, dovrà essere inviato all’indirizzo Pec del Ministero 
della Giustizia offertapvp.dgsia@giustiziacert.it. Saranno dichiarate inammissibili le offerte 
provenienti da “presentatori” diversi dai soggetti suindicati (offerenti o procuratore 
legale). 
A pena d’inammissibilità l’offerta dovrà essere sottoscritta digitalmente utilizzando un 
certificato di firma digitale in corso di validità rilasciato da un organismo incluso nell’elenco 
pubblico dei certificatori accreditati e trasmessa a mezzo casella di posta elettronica certificata 
(dell’offerente o del suo procuratore legale); in alternativa è possibile trasmettere l’offerta e gli 
allegati a mezzo casella di posta elettronica certificata per la vendita telematica ai sensi dell’art. 
12, comma 5 del D.M. n. 32/20153. 
L’offerta si intenderà depositata nel momento in cui viene generata la ricevuta completa di 
 
4. Il responsabile per i sistemi informativi automatizzati del Ministero verifica, su richiesta dei gestori di cui al 
comma 1, che il procedimento previsto per il rilascio delle credenziali di accesso sia conforme a quanto previsto dal 
presente articolo e li iscrive in un'apposita area pubblica del portale dei servizi telematici del Ministero. 
Art. 14  
DEPOSITO E TRASMISSIONE DELL'OFFERTA AL GESTORE PER LA VENDITA TELEMATICA 
1. L'offerta si intende depositata nel momento in cui viene generata la ricevuta completa di avvenuta consegna da 
parte del gestore di posta elettronica certificata del ministero della giustizia. 
2. L'offerta pervenuta all'indirizzo di posta elettronica certificata di cui all'articolo 13, comma 1, è automaticamente 
decifrata non prima di centottanta e non oltre centoventi minuti antecedenti l'orario fissato per l'inizio delle 
operazioni di vendita. 
3. Il software di cui all'articolo 12, comma 3, elabora un ulteriore documento testuale, privo di restrizioni per le 
operazioni di selezione e copia, in uno dei formati previsti dalle specifiche tecniche dell'articolo 26. Il documento 
deve contenere i dati dell'offerta, salvo quelli di cui all'articolo 12, comma 1, lettere a), n) ed o). 
4. L'offerta e il documento di cui al comma 2 sono trasmessi ai gestori incaricati delle rispettive vendite nel rispetto 
del termine di cui al comma 1. 
Art. 15  
MANCATO FUNZIONAMENTO DEI SERVIZI INFORMATICI DEL DOMINIO GIUSTIZIA 
1. Il responsabile per i sistemi informativi automatizzati del ministero comunica preventivamente ai gestori della 
vendita telematica i casi programmati di mancato funzionamento dei sistemi informativi del dominio giustizia. I 
gestori ne danno notizia agli interessati mediante avviso pubblicato sui propri siti internet e richiedono di pubblicare 
un analogo avviso ai soggetti che gestiscono i siti internet ove è eseguita la pubblicità di cui all'articolo 490 del 
codice di procedura civile. Nei casi di cui al presente comma le offerte sono formulate a mezzo telefax al recapito 
dell'ufficio giudiziario presso il quale è iscritta la procedura, indicato negli avvisi di cui al periodo precedente. Non 
prima del giorno precedente l'inizio delle operazioni di vendita il gestore ritira le offerte formulate a norma del 
presente comma dall'ufficio giudiziario. 
2. Nei casi di mancato funzionamento dei sistemi informativi del dominio giustizia non programmati o non 
comunicati a norma del comma 1, l'offerta si intende depositata nel momento in cui viene generata la ricevuta di 
accettazione da parte del gestore di posta elettronica certificata del mittente. Il gestore è tenuto a permettere la 
partecipazione alle operazioni di vendita dell'offerente che documenta la tempestiva presentazione dell'offerta a 
norma del periodo precedente. 
 
2 Il pagamento del bollo potrà essere effettuato solo con modalità telematica, a mezzo bonifico o carta di credito, 
accedendo all’area “Pagamento di bolli digitali” sul Portale dei Servizi Telematici, all’indirizzo 
https://pst.giustizia.it. 
3 Detta modalità di trasmissione dell’offerta sarà operativa una volta che saranno eseguite a cura del Ministero della 
giustizia le formalità di cui all’art. 13, comma 4, D.M. n. 32/15; 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
11 
 
avvenuta consegna da parte del gestore Pec del Ministero della Giustizia;  
In caso di mancato funzionamento dei servizi telematici del sistema giustizia l’offerta dovrà 
essere depositata ai sensi dell’art. 15 D.M. 
2. CONTENUTO DELL'OFFERTA 
L’offerta dovrà contenere: 
a) 
il cognome, il nome, il luogo, la data di nascita, il codice fiscale o partita IVA, il 
domicilio, lo stato civile, ed il recapito telefonico del soggetto cui andrà intestato l’immobile 
(non sarà possibile intestare l’immobile a soggetto diverso da quello che sottoscrive l’offerta). Se 
l’offerente è coniugato in regime di comunione legale dei beni, dovranno essere indicati anche i 
corrispondenti dati del coniuge. Se l’offerente è minorenne, l’offerta dovrà essere sottoscritta dai 
genitori previa autorizzazione del giudice tutelare; 
b) 
l’ufficio giudiziario presso il quale pende la procedura; 
c) 
l’anno e il numero di ruolo generale della procedura; 
d) 
il numero o altro dato identificativo del lotto; 
e) 
la descrizione del bene; 
f) 
l’indicazione del referente della procedura/delegato alle operazioni di vendita; 
g) 
la data e l’ora fissata per l’inizio delle operazioni di vendita; 
h) 
il prezzo offerto, che non potrà essere, a pena di inefficacia dell’offerta, inferiore di oltre 
un quarto al valore d’asta indicato nella presente ordinanza e quindi nell’avviso di vendita;  
i) 
il termine di pagamento del saldo prezzo e degli oneri accessori, che non potrà comunque 
essere superiore a centoventi giorni dalla data dell’aggiudicazione (termine soggetto a 
sospensione nel periodo feriale); 
l) 
l’importo versato a titolo di cauzione, in misura pari o superiore al 10 per cento del 
prezzo offerto, importo che sarà trattenuto in caso di rifiuto dell’acquisto; 
m) 
la data, l’orario e il numero di CRO del bonifico effettuato per il versamento della 
cauzione4; 
n) 
il codice IBAN del conto sul quale è stata addebitata la somma oggetto del bonifico; 
o) 
l’indirizzo della casella di posta elettronica certificata (o della casella di posta elettronica 
certificata per la vendita telematica) utilizzata per trasmettere l’offerta e per ricevere le 
comunicazioni previste ed il recapito di telefonia mobile ove ricevere le eventuali comunicazioni 
previste5; 
p) l’espressa dichiarazione di aver preso visione della perizia di stima; 
NB: Le dichiarazioni ed i dati richiesti non previsti dal modulo ministeriale dovranno essere 
oggetto di separata dichiarazione da allegare telematicamente all’offerta. 
3. DOCUMENTI DA ALLEGARE ALL’OFFERTA 
a) 
copia del documento di identità dell’offerente e del coniuge in comunione legale dei beni; 
b) 
copia del bonifico attestante il versamento della cauzione; in difetto – laddove la cauzione 
non potesse essere abbinata all’offerta o rinvenuta sul conto dedicato (ad es. per errori di 
digitazione del CRO) – l’offerta sarà dichiarata inammissibile; 
 
4 Nel caso in cui nell’attestazione di avvenuto pagamento venga indicato come codice identificativo della 
transazione il c.d. codice TRN (o codice similare) composto da 30 caratteri alfanumerici, il CRO è normalmente 
identificato dalle 11 cifre presenti tra il 6° ed il 16° carattere del TRN. Non devono essere indicati caratteri diversi 
da numeri o inseriti spazi, caratteri speciali o trattini. 
5 Sia la casella di Posta Elettronica Certificata che il recapito di telefonia mobile devono appartenere 
all’offerente o al procuratore legale depositante. 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
12 
 
c) 
se l’offerta è formulata da più persone, copia anche per immagine della procura rilasciata 
dagli altri offerenti per atto pubblico o scrittura privata autenticata in favore del soggetto titolare 
della casella di posta elettronica certificata per la vendita telematica oppure del soggetto che 
sottoscrive l’offerta, laddove questa venga trasmessa a mezzo di casella di posta elettronica 
certificata; 
d) 
copia anche per immagine della procura speciale o della procura generale, nell’ipotesi di 
offerta fatta a mezzo di procuratore legale, cioè di avvocato;  
e) 
copia della visura camerale della società attestante i poteri del legale rappresentante della 
persona giuridica offerente, risalente a non più di tre mesi, ovvero copia della delibera 
assembleare che autorizzi un soggetto interno alla società alla partecipazione alla vendita in 
luogo del legale rappresentante e originale della procura speciale o copia autentica della procura 
generale rilasciate da questi attestanti i poteri del soggetto interno delegato; 
f) 
se il soggetto offerente è minorenne, copia del documento d’identità e copia del codice 
fiscale del soggetto offerente e del soggetto che sottoscrive l’offerta, nonché copia del 
provvedimento di autorizzazione; 
g) 
copia - anche per immagine della dichiarazione del coniuge - autenticata da pubblico 
ufficiale - di voler escludere il bene dalla comunione legale, in caso di acquisto del bene da parte 
dell’altro coniuge a titolo personale; 
h) 
le dichiarazioni relative allo stato civile, di aver preso visione della perizia di stima etc., 
effettuate in separato modulo; 
i) 
ricevuta di pagamento del bollo effettuata in via telematica.6 
 
4. MODALITÀ DI VERSAMENTO DELLA CAUZIONE 
Esclusivamente con bonifico sul conto corrente cauzioni intestato al Tribunale di Roma, con il 
seguente codice IBAN IT83P0832703399000000310104; e con unica causale “ASTA” (senza 
ulteriori specificazioni di dati identificativi della procedura); la cauzione, a pena di inefficacia 
della offerta, dovrà risultare accreditata sul conto entro il giorno precedente alla vendita e dovrà 
essere di importo pari almeno al 10 per cento del prezzo offerto. 
5. IRREVOCABILITÀ DELL'OFFERTA 
Salvo quanto previsto dall’art. 571 c.p.c., l’offerta presentata nella vendita senza incanto è 
irrevocabile. Si potrà procedere all’aggiudicazione al maggior offerente anche qualora questi non 
si colleghi telematicamente il giorno fissato per la vendita; 
6. SVOLGIMENTO DELLA VENDITA 
Le buste telematiche saranno aperte all’orario di inizio delle operazioni di vendita, alla presenza 
eventuale delle parti, dei loro avvocati e dei creditori iscritti non intervenuti, il giorno ed all’ora 
indicati nell’avviso di vendita; gli offerenti parteciperanno in via telematica alle operazioni di 
vendita collegandosi tramite l’area riservata del sito del gestore della vendita telematica, 
accedendo alla stessa con le credenziali personali a loro trasmesse almeno 30 minuti prima 
dell’inizio delle operazioni di vendita alla casella di posta elettronica certificata (ovvero alla 
casella di posta elettronica certificata per la vendita telematica) utilizzata per trasmettere 
l’offerta. 
Il ritardo sui tempi indicati nell’avviso per l’inizio delle operazioni di apertura buste o di vendita 
non può costituire motivo di doglianza da parte degli offerenti. 
 
6 La mancata allegazione della ricevuta telematica del pagamento del bollo non determina l’inammissibilità 
dell’offerta ma l’avvio delle procedure di recupero coattivo. 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
13 
 
Il professionista delegato, referente della procedura, verificata la regolarità delle offerte darà 
inizio alle operazioni di vendita. 
Qualora per l’acquisto del medesimo bene siano state proposte più offerte valide, si procederà a 
gara sulla base della offerta più alta, che avrà durata di 24 ore a partire dal suo inizio, con 
proroga automatica alla stessa ora del giorno successivo non festivo qualora il suo termine 
venisse a cadere di sabato o in giorni festivi; durante il periodo della gara, ogni partecipante 
potrà effettuare offerte in aumento, cioè rilanci, nel rispetto dell’importo minimo stabilito 
dall’avviso di vendita a pena di inefficacia; qualora vengano effettuate offerte in aumento negli 
ultimi 10 minuti della gara la stessa sarà prorogata automaticamente di ulteriori 10 minuti in 
modo da permettere agli altri partecipanti di effettuare ulteriori rilanci e così di seguito sino alla 
mancata presentazione di offerte in aumento nel periodo di prolungamento ma comunque fino ad 
un massimo di dodici prolungamenti (quindi per un totale di ulteriori 120 minuti); la 
deliberazione finale sulle offerte all’esito della gara avrà luogo entro il giorno successivo al suo 
termine, prorogata se cadente di sabato o festivi al primo giorno non festivo; il bene verrà 
definitivamente aggiudicato dal professionista delegato referente della procedura, facendosi così 
luogo alla vendita, a chi avrà effettuato l’offerta più alta, sulla base delle comunicazioni 
effettuate dal gestore della vendita telematica. In difetto di offerte in aumento, il professionista 
delegato referente della procedura disporrà la vendita a favore del migliore offerente (come 
individuato al precedente n. 12), salvo che il prezzo offerto sia inferiore al valore d’asta stabilito 
nell’avviso di vendita e vi siano istanze di assegnazione ovvero, in loro difetto, non sussistano i 
presupposti di cui all’art. 572, 3° c.p.c.; all’offerente che non risulterà aggiudicatario la cauzione 
sarà restituita mediante bonifico sul conto corrente dal quale era stata inviata. 
7. TERMINI PER IL SALDO PREZZO 
In caso di aggiudicazione il termine per il deposito del saldo del prezzo e delle spese (soggetto a 
sospensione nel periodo feriale) sarà comunque quello indicato nell’offerta dall’aggiudicatario. 
Nell’ipotesi in cui non sia stato indicato il termine ovvero sia stato indicato un termine superiore 
a 120 giorni, il versamento del saldo del prezzo e delle spese dovrà essere effettuato comunque 
entro detto termine. 
8. OFFERTA PER PERSONA DA NOMINARE 
Nell’ipotesi in cui il procuratore legale, cioè avvocato, abbia effettuato l’offerta e sia rimasto 
aggiudicatario per persona da nominare, dovrà dichiarare al delegato nei tre giorni successivi alla 
vendita il nome della persona per la quale ha fatto l’offerta, depositando originale della procura 
speciale notarile, ovvero copia autentica della procura generale, rilasciate in data non successiva 
alla vendita stessa, ovvero trasmettendogli via PEC detta documentazione in copia per immagine 
con attestazione di conformità. 
9. ISTANZA DI ASSEGNAZIONE CON RISERVA DI NOMINA DI UN TERZO 
Il creditore che è rimasto assegnatario a favore di un terzo dovrà dichiarare al delegato, nei 
cinque giorni dal provvedimento di assegnazione, il nome del terzo a favore del quale deve 
essere trasferito l'immobile, depositando la dichiarazione del terzo di volerne profittare, con 
sottoscrizione autenticata da pubblico ufficiale e i documenti comprovanti gli eventualmente 
necessari poteri ed autorizzazioni. In mancanza, il trasferimento è fatto a favore del creditore. In 
ogni caso, gli obblighi derivanti dalla presentazione dell'istanza di assegnazione sono 
esclusivamente a carico del creditore.  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
14 
 
[B] 
 
DISPOSIZIONI RELATIVE AL PAGAMENTO  
DEL PREZZO E DEGLI ONERI ACCESSORI 
1. 
Salvo quanto disposto sub punto 2), il saldo del prezzo di aggiudicazione (pari al prezzo 
di aggiudicazione dedotta la cauzione prestata) dovrà essere versato mediante bonifico sul conto 
corrente di pertinenza della procedura esecutiva entro il termine indicato nell’offerta, ovvero, 
qualora sia stato indicato un termine superiore, entro il termine massimo di 120 giorni dalla data 
di vendita (soggetto a sospensione nel periodo feriale); 
2. 
qualora l'immobile sia gravato da ipoteca iscritta a garanzia di mutuo concesso ai sensi 
del T.U. 16 luglio 1905, n. 646, richiamato dal D.P.R. 21 gennaio 1976 n. 7 ovvero ai sensi 
dell'art. 38 del Decreto legislativo 10 settembre 1993 n. 385 (mutuo fondiario), l'aggiudicatario 
dovrà versare direttamente all'Istituto mutuante, nel termine per il versamento del saldo del 
prezzo, la parte del prezzo corrispondente al credito dell'Istituto per capitale, interessi, accessori 
e spese di procedura nei limiti di quanto disposto dall’art.2855 c.c.., così come individuata dal 
delegato. L'eventuale somma residua dovrà essere versata con le modalità indicate sub punto 1); 
l’aggiudicatario dovrà tempestivamente consegnare al delegato l’originale della quietanza 
rilasciata dall’istituto di credito; 
3. 
qualora l’aggiudicatario, per il pagamento del saldo prezzo faccia ricorso ad un contratto 
bancario di finanziamento con concessione di ipoteca di primo grado sull’immobile acquistato, le 
somme dovranno essere erogate nel termine fissato per il versamento del saldo prezzo, 
direttamente dall’istituto di credito mutuante con le modalità indicate dal delegato mediante 
bonifico sul conto corrente di pertinenza della procedura esecutiva, anche nell’ipotesi di cui al 
punto precedente. L’aggiudicatario dovrà quindi consegnare al delegato, l’atto di assenso ad 
iscrizione di ipoteca con la relativa nota di iscrizione e ricevuta di avvenuta registrazione 
dell’atto. Contestualmente alla presentazione del decreto sottoscritto per la trascrizione, il 
delegato presenterà al Conservatore dei Registri Immobiliari anche la nota di iscrizione di 
ipoteca. Conformemente a quanto previsto dall’art. 585, ultimo comma, c.p.c., il delegato nel 
predisporre la minuta del decreto di trasferimento da sottoporre alla firma del Giudice 
dell’esecuzione inserirà la seguente dizione: “rilevato che il pagamento di parte del prezzo 
relativo al trasferimento del bene oggetto del presente decreto è avvenuto mediante erogazione 
della somma di €***** da parte di **** a fronte del contratto di mutuo a rogito **** del **** 
rep.*** e che le parti mutuante e mutuataria hanno espresso il consenso all’iscrizione di ipoteca 
di primo grado a garanzia del rimborso del predetto finanziamento, si rende noto che, 
conformemente a quanto disposto dall’art.585 c.p.c., è fatto divieto al Conservatore dei RR.II. di 
trascrivere il presente decreto se non unitamente all’iscrizione dell’ipoteca di cui all’allegata 
nota”. In caso di revoca dell’aggiudicazione, e sempre che l’Istituto mutuante ne abbia fatto 
ESPRESSA richiesta con atto ritualmente depositato in cancelleria e trasmesso in copia al 
delegato, le somme erogate potranno essere restituite direttamente all’Istituto di credito senza 
aggravio di spese per la procedura; 
4. 
nel medesimo termine fissato per il versamento del saldo del prezzo, l’aggiudicatario è 
tenuto anche al pagamento degli oneri fiscali e tributari conseguenti all’acquisto del bene, 
nonché dei compensi spettanti a norma del decreto ministeriale n. 227 del 2015 al delegato 
incaricato della registrazione, trascrizione e voltura del decreto di trasferimento e della 
cancellazione delle formalità pregiudizievoli. Egli è del pari tenuto ad anticipare gli importi delle 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
15 
 
imposte di cancellazione delle formalità pregiudizievoli. L’importo complessivo di tali oneri 
accessori sarà tempestivamente comunicato dal delegato successivamente all’aggiudicazione. Il 
delegato, unitamente al deposito della minuta del decreto di trasferimento, depositerà apposita 
certificazione relativa all’esatto e tempestivo versamento da parte dell’aggiudicatario delle spese 
e degli oneri accessori. Le somme a qualunque titolo versate dall’aggiudicatario saranno 
preventivamente imputate al pagamento degli oneri accessori e delle spese e, quindi, al 
pagamento del saldo del prezzo. Il mancato versamento entro il termine di quanto 
complessivamente dovuto (sia a titolo di saldo del prezzo, sia a titolo di oneri accessori) 
comporterà la decadenza dall’aggiudicazione e la perdita della cauzione. 
 
DISCIPLINA DELLA PUBBLICITÀ DELLE VENDITE DELEGATE 
della vendita sarà data pubblica notizia: 
a) dal professionista delegato mediante l’inserimento sul Portale delle vendite pubbliche 
dell’avviso di vendita e della ordinanza di delega; 
b) dal custode mediante pubblicazione della presente ordinanza, dell’avviso e dell’elaborato 
peritale posto alla base della vendita (completo di planimetrie, fotografie e degli altri allegati 
eventualmente indicati dal custode), almeno 45 giorni prima del termine per la presentazione 
delle offerte e fino al giorno delle vendite sul sito internet del gestore della vendita nominato7 
nonché sul sito www.tribunale.roma.it.; 
c) dal custode sarà pubblicata, altresì, la documentazione di cui al punto b) sui siti degli altri 
gestori in rotazione presso il Tribunale di Roma al fine di garantire la maggior diffusione degli 
annunci e la completezza e uniformità delle informazioni per gli utenti; 
d) dal custode mediante pubblicazione sui siti internet Casa.it e Idealista.it per il periodo di 
almeno 45 giorni prima della data fissata per la vendita senza incanto, di un avviso contenente in 
modo sintetico e riassuntivo la descrizione dell’immobile posto in vendita ed il valore d’asta con 
l’offerta minima, assieme alla indicazione che si tratta di vendita giudiziaria, che è l’unico 
annuncio autorizzato per tale vendita e che non sono dovuti compensi né oneri aggiuntivi di 
alcun tipo, assieme al link ovvero alla indicazione dei siti Internet e www.tribunale.roma.it ove 
saranno disponibili in dettaglio tutte le altre necessarie informazioni, pure in forma sintetica nella 
“ scheda riassuntiva ”. 
AFFIDA 
L’incarico di provvedere a tutte le attività necessarie alla pubblicazione delle vendite disposte 
con la presente ordinanza ai precedenti punti b e c alla società nominata gestore della vendita la 
quale dovrà quindi provvedere, previo avvenuto pagamento dei relativi costi: 
a. a curare, su ordine del custode, la pubblicazione dell’avviso di vendita, dell’ordinanza di 
delega e dell’elaborato peritale di stima sul proprio sito internet, nonché su quello degli altri 
gestori in rotazione presso l’ufficio e sul sito istituzionale del Tribunale di Roma 
(www.tribunale.roma.it); nonché (nei modi e nei tempi su indicati) a pubblicare l’avviso sui siti 
Internet Casa.it e Idealista.it, tramite il sistema “Rete Aste Real Estate”; 
b. ad acquisire direttamente dagli uffici di Cancelleria e facendo ricorso esclusivamente al 
personale espressamente autorizzato dal Tribunale, la copia (anche in formato elettronico) di 
 
7 I siti internet dei gestori in rotazione temperata presso il Tribunale di Roma alla data di emissione della presente 
delega sono: www.astegiudiziarie.it, www.asteannunci.it, www.fallcoaste.it  
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
16 
 
tutta la documentazione di cui è stata disposta o si renderà necessaria la pubblicazione; 
c. a rendere disponibile, sia al custode che al delegato, tramite l’area riservata del proprio sito 
almeno una settimana prima della data fissata per la vendita, adeguata documentazione attestante 
lo svolgimento degli adempimenti pubblicitari; 
d. il professionista delegato provvederà ad acquisire il giorno precedente a quello fissato per la 
vendita, tramite l’area riservata del Portale delle Vendite Pubbliche, la certificazione relativa 
all’inizio della pubblicazione delle singole inserzioni pubblicitarie sul PVP e sui siti di pubblicità 
legale in base all’art. 490 c.2 c.p.c., alla loro durata ed agli eventi significativi, in base all’art. 7 
del D.M. 31/10/2006; 
DISPONE 
che il Custode: 
a. verificata la procedibilità dell’azione esecutiva ed invitati, se necessario, i creditori al 
pagamento degli oneri della pubblicità, diversi dal contributo di pubblicazione sul Portale delle 
vendite pubbliche, provveda ad ordinare al gestore della vendita la pubblicazione mediante invio, 
almeno venti giorni prima del termine ultimo per la pubblicazione sui siti internet (65 giorni 
prima della vendita senza incanto), dell’avviso di vendita del delegato unitamente ad apposito 
modulo di richiesta pubblicità. Tali documenti dovranno essere trasmessi esclusivamente tramite 
posta elettronica tramite le apposite funzionalità di “invio telematico” disponibili nelle aree 
riservate dei siti dei gestori o tramite posta elettronica8; 
b. indichi di quali allegati all’elaborato peritale si debba dare pubblicazione sui siti internet; 
c. provveda alla verifica della correttezza dei dati inseriti sul Portale delle vendite pubbliche e in 
ogni avviso pubblicato, anche via internet, segnalando tempestivamente e direttamente al 
delegato e al gestore della pubblicità eventuali errori, omissioni o imprecisioni; 
d. comunichi al più presto l’avviso di vendita al gestore della vendita, ai creditori di cui all’art. 
498 c.p.c. che non sono comparsi, alle parti esecutate, alle parti debitrici qualora la procedura si 
svolga nelle forme dell’art.602 e ss. c.p.c.; 
e. invii al delegato a mezzo pec almeno 7 giorni prima della vendita copia conforme delle 
comunicazioni dell’avviso di vendita e della ordinanza di delega; 
f. assicuri agli interessati all’acquisto, che ne abbiano fatto tempestiva richiesta a mezzo del 
Portale delle vendite pubbliche, di visitare l’immobile nella dovuta riservatezza e nel rispetto di 
eventuali prescrizioni di natura sanitaria in tempo utile per il deposito dell’offerta, 
accompagnandoli di persona o tramite un proprio delegato (senza costi aggiuntivi), precisando 
gli eventuali oneri condominiali per i quali, in base a quanto disposto dall’art.63, 2° comma, 
disp. att. c.c., il potenziale aggiudicatario potrebbe essere chiamato a rispondere solidamente con 
il debitore, nonché lo stato occupativo dell’immobile e la natura di titolo per l’esecuzione forzata 
per rilascio del decreto di trasferimento. g. attuare la liberazione in favore dell’aggiudicatario, su 
richiesta del medesimo, ai sensi del novellato art. 560 VI co.; 
g. Vigili sul rispetto dei provvedimenti del garante per la protezione dei dati personali del 22 
ottobre 1998 e del 07 febbraio 2008 e del Regolamento (UE) 2016/679. 
P O N E 
gli ulteriori oneri economici necessari alla pubblicità, (diversa dalla pubblicazione sul Portale 
delle vendite pubbliche), qualora non vi siano somme sufficienti a disposizione della procedura, 
 
8 Alla società Aste Giudiziarie Inlinea S.p.A. (tel. 058620141, email: pubblicazione@astegiudiziarie.it); alla società 
Edicom Finance S.r.l. (tel. 0415369911, email: info.roma@edicomsrl.it); alla società Zucchetti Software Giuridico 
S.r.l. (tel. 0444346211, email: aste@fallco.it) 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
17 
 
a carico anticipato provvisorio di tutti i creditori, avvertendoli che la mancata tempestiva 
corresponsione delle somme agli ausiliari della pubblicità impedirà i relativi adempimenti e che 
se ciò accadrà per due volte l’esecuzione verrà dichiarata improcedibile. 
 
AUTORIZZA 
il Gestore della vendita designato ad accedere al conto cauzioni per gli adempimenti allo stesso 
affidati. 
R E N D E    N O T O 
a) che la vendita avviene nello stato di fatto e di diritto in cui i beni si trovano (anche in relazione 
al testo unico di cui al decreto del Presidente della Repubblica 6 giugno 2001, n. 380) con tutte le 
eventuali pertinenze, accessioni, ragioni ed azioni, servitù attive e passive. La vendita è a corpo e 
non a misura. Eventuali differenze di misura non potranno dar luogo ad alcun risarcimento, 
indennità o riduzione del prezzo; 
b) che la vendita forzata non è soggetta alle norme concernenti la garanzia per vizi o mancanza 
di qualità, né potrà essere revocata per alcun motivo. Conseguentemente, l’esistenza di eventuali 
vizi, mancanza di qualità o difformità della cosa venduta, oneri di qualsiasi genere ivi compresi, 
ad esempio, quelli urbanistici ovvero derivanti dalla eventuale necessità di adeguamento di 
impianti alle leggi vigenti, spese condominiali dell’anno in corso e dell’anno precedente non 
pagate dal debitore, per qualsiasi motivo non considerati, anche se occulti e comunque non 
evidenziati in perizia, non potranno dar luogo ad alcun risarcimento, indennità o riduzione del 
prezzo, essendosi di ciò tenuto conto nella valutazione dei beni; 
c) che per gli immobili realizzati in violazione della normativa urbanistico edilizia, 
l’aggiudicatario, potrà ricorrere, ove consentito, alla disciplina dell’art.40 della legge 28 febbraio 
1985, n.47 come integrato e modificato dall’art.46 del D.P.R. 6 giugno 2001, n.380, purché 
presenti domanda di concessione o permesso in sanatoria entro 120 giorni dalla notifica del 
decreto di trasferimento; 
d) che l’immobile viene venduto libero da iscrizioni ipotecarie e da trascrizioni di pignoramenti e 
sequestri civili che saranno cancellate a cura e spese della procedura; 
e) che l’aggiudicatario ha facoltà di esonerare il custode dall’attuazione dell’ordine di liberazione 
emesso prima dell’aggiudicazione con dichiarazione da far pervenire a mezzo mail e/o PEC al 
custode in tempo utile; avvenuta l’aggiudicazione, l’ordine di liberazione, per i casi diversi da 
quello di cui all’art. 560, 3° co. c.p.c., sarà emesso solo su istanza dell’aggiudicatario da far 
pervenire al custode con le forme di cui al periodo che precede; in ogni caso, all’atto del 
pagamento del saldo prezzo, l’aggiudicatario o l’assegnatario potrà avanzare richiesta di 
attuazione a spese della procedura del provvedimento di cui all'articolo 586, secondo comma 
c.p.c. con istanza a mezzo mail e/o PEC al professionista delegato, a cui si darà corso a mezzo 
del custode giudiziario, previa notifica o comunicazione del decreto di trasferimento completo 
delle modalità stabilite dal Giudice per la chiesta attuazione ex art. 560 c.p.c, ultimo comma ai 
periodi dal secondo al settimo, decorsi sessanta giorni e non oltre centoventi giorni; in difetto di 
tempestiva richiesta dell’aggiudicatario, l’ingiunzione di rilascio contenuta nel decreto di 
trasferimento, emesso privo della apposita disciplina dell’attuazione ex art. 560 c.p.c. potrà 
essere eseguito esclusivamente nelle forme di cui all’art. 605 c.p.c. a cura dell’aggiudicatario o 
dell’assegnatario. 
f) che l’elaborato peritale relativo alla stima e valutazione degli immobili posti in vendita con la 
presente ordinanza sarà disponibile per la consultazione sul sito internet dei gestori della vendita 
e www.tribunale.roma.it; 
Avviso di vendita delega telematica asincrona 2020 
[Rev 25/06/2020] 
18 
 
g) che la partecipazione alle vendite giudiziarie non esonera gli offerenti dal compiere le visure 
ipotecarie e catastali e dall’accertare il regime fiscale applicabile all’acquisto; 
h) che maggiori informazioni, anche relative alle generalità del soggetto passivo della procedura 
esecutiva, possono essere fornite dal custode in epigrafe indicato a chiunque vi abbia interesse. 
A V V E R T E 
a) che in base a quanto disposto dall’art.624 bis c.p.c., il Giudice dell’esecuzione, può, sentito il 
debitore, sospendere, per una sola volta, il processo fino a ventiquattro mesi, su istanza di tutti i 
creditori muniti di titolo esecutivo. L’istanza può essere proposta fino a venti giorni prima della 
scadenza del termine per il deposito delle offerte d’acquisito nel caso di vendita senza incanto, o 
fino a quindici giorni prima dell’incanto; 
b) che, in base a quanto disposto dall’art.161 bis disp. att. c.p.c., il rinvio della vendita può essere 
disposto solo con il consenso dei creditori e degli offerenti che abbiano prestato cauzione ai sensi 
degli articoli 571 e 580 c.p.c. 
I N F O R M A 
gli offerenti che, per partecipare alle vendite giudiziarie, non è necessaria alcuna forma di 
intermediazione e che, eventuali chiarimenti e delucidazioni, potranno essere richieste al custode 
giudiziario, al delegato alla vendita ed al gestore della vendita; quest’ultimo potrà fornire 
assistenza alla compilazione e deposito dell’offerta al numero di help desk: 0444/346211, a mezzo 
mail: aste@fallco.it o presso la sala d'aste suindicata. 
 
Roma, 11 novembre 2020 
 
 
Il Professionista Delegato 
Notaio Carlo Gaddi """

Prova estrazione dizionario mappato:

In [16]:
key_list = 'mq', 'metriquadri'
dictr = get_text_and_values(house, key_list)

print(dictr)

{'Piazza Ener Bettica n. 26, e precisamente: \n- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una \nsuperficie di mq. 44 circa a confine con  Piazza Ener Bettica, negozio sub. 4, viale condominiale, \nsalvo altri.': [44], 'Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. \n1305, Sub. 5, Zc. 7, Categoria C1, cl. 8, mq. 27, Piazza Ener Bettica n° 26, P.T, int. 1, rendita catastale \nEuro 715,34.': [8, 27], 'Piazza Ener Bettica n. 27/28, Via Carlo Bosio n° 117-119 e precisamente: \n- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una \nsuperficie di mq. 92 circa a confine con  Piazza Ener Bettica, negozio sub. 4, Via Carlo Bosio, salvo \naltri.': [92], 'Detta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. \n1305, Sub. 4, Zc. 7, Categoria C1, cl. 7, mq. 92, Piazza Ener Bettica n° 26-28 Via Carlo Bosio n

Prova estrazione testo di interesse:

In [17]:
get_useful_text(dictr)

'Piazza Ener Bettica n. 26, e precisamente: \n- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una \nsuperficie di mq. 44 circa a confine con  Piazza Ener Bettica, negozio sub. 4, viale condominiale, \nsalvo altri.\nDetta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. \n1305, Sub. 5, Zc. 7, Categoria C1, cl. 8, mq. 27, Piazza Ener Bettica n° 26, P.T, int. 1, rendita catastale \nEuro 715,34.\nPiazza Ener Bettica n. 27/28, Via Carlo Bosio n° 117-119 e precisamente: \n- Negozio sito al piano terra dell’edificio E, con annessi bagno, antibagno e ripostiglio per una \nsuperficie di mq. 92 circa a confine con  Piazza Ener Bettica, negozio sub. 4, Via Carlo Bosio, salvo \naltri.\nDetta Porzione Immobiliare è censita al catasto Fabbricati del Comune di Roma al Foglio 1079, Part. \n1305, Sub. 4, Zc. 7, Categoria C1, cl. 7, mq. 92, Piazza Ener Bettica n° 26-28 Via Carlo Bosio n° 117-\n119 P.T, int. 2, ren

prova estrazione valori senza transformer:

In [18]:
get_values(dictr)

[[44],
 [8, 27],
 [92],
 [7, 92],
 [16],
 [3, 16],
 [12],
 [3, 11],
 [58],
 [6, 58],
 [107],
 [6, 107],
 [114],
 [6, 115],
 [114],
 [6, 106],
 [58],
 [6, 58],
 [53],
 [7, 55],
 [80],
 [7, 70],
 [50],
 [7, 50],
 [61],
 [7, 64]]

prova estrazione valori con transformer:

In [19]:
model = 'it5/it5-base-question-answering'
qa = initialize_qa_transformer(model)

In [20]:
get_answers_unfiltered(dictr) #con duplicati

[[{'generated_text': '44'}],
 [{'generated_text': '27'}],
 [{'generated_text': '92'}],
 [{'generated_text': '92'}],
 [{'generated_text': '16'}],
 [{'generated_text': '16'}],
 [{'generated_text': '12'}],
 [{'generated_text': '11'}],
 [{'generated_text': '58'}],
 [{'generated_text': '58'}],
 [{'generated_text': '107'}],
 [{'generated_text': '107'}],
 [{'generated_text': '114'}],
 [{'generated_text': '115'}],
 [{'generated_text': '114'}],
 [{'generated_text': '106'}],
 [{'generated_text': '58'}],
 [{'generated_text': '58'}],
 [{'generated_text': '53'}],
 [{'generated_text': '55'}],
 [{'generated_text': '80'}],
 [{'generated_text': '70'}],
 [{'generated_text': '50'}],
 [{'generated_text': '50'}],
 [{'generated_text': '61'}],
 [{'generated_text': '64'}]]

In [21]:
val = get_answers_filtered(dictr)
val  #senza duplicati, valori reali

[[{'generated_text': '44'}],
 [{'generated_text': '27'}],
 [{'generated_text': '92'}],
 [{'generated_text': '16'}],
 [{'generated_text': '12'}],
 [{'generated_text': '11'}],
 [{'generated_text': '58'}],
 [{'generated_text': '107'}],
 [{'generated_text': '114'}],
 [{'generated_text': '115'}],
 [{'generated_text': '114'}],
 [{'generated_text': '106'}],
 [{'generated_text': '58'}],
 [{'generated_text': '53'}],
 [{'generated_text': '55'}],
 [{'generated_text': '80'}],
 [{'generated_text': '70'}],
 [{'generated_text': '50'}],
 [{'generated_text': '61'}],
 [{'generated_text': '64'}]]

Prova estrazione totale mq

In [27]:
keywords = ['totale', 'complessivo', 'complessiva']
get_total(answered_values= val, text = house, keywords = keywords) 

1307

N:B : Implementare Spacy sentence segmentation al posto dell'algoritmo con regex